# Explore population density in the bay area

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering


# Modules and Packages



In [2]:
import math
import numpy as np
import pandas as pd

import psycopg2

from geographiclib.geodesic import Geodesic

# Supporting code

In [3]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [10]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    return(pd.DataFrame(cursor.fetchall(), columns =['zip', 'population']))
        

In [5]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [6]:
cursor = connection.cursor()

# Find all zip codes and population, within 1, 3, 5, and 10 miles of the Downtown Berkeley station

To start, we'll look at all of the zip codes and population of those zip codes that we can service from the Downtown Berkley station. We'll use these panda's dataframes to compare to other stations to ensure that we're reaching net new cusotmers.

In [23]:
downtown_berkley_1_mile = my_station_get_zips('Downtown Berkeley', 1)
downtown_berkley_3_mile = my_station_get_zips('Downtown Berkeley', 3)
downtown_berkley_5_mile = my_station_get_zips('Downtown Berkeley', 5)
downtown_berkley_10_mile = my_station_get_zips('Downtown Berkeley', 10)

downtown_berkley_1_mile

,zip,population
0,94702,17092
1,94703,21937
2,94704,29190
3,94709,11740
4,94720,2971


# Get list of all other stations excluding 'Downtown Berkley'

In [22]:
def get_stations():
    connection.rollback()
    
    query = "select station from stations where station <> 'Downtown Berkley'"

    cursor.execute(query)

    stations = []
    temp_stations = cursor.fetchall()
    for station in temp_stations:
        stations.append(station[0])
    
    return stations

# Fin all zip codes and population within a distence of every station

At least 75% of the population must be in zip codes that are not within the same distence of the Downtown Berkeley station

In [62]:
def find_net_new_customers(distance):
    """finds all stations where 50% of the population within the distence would be net new"""
    stations = get_stations()
    
    berkeley_zips = my_station_get_zips('Downtown Berkeley', distance)
    berkeley_pop = berkeley_zips['population'].sum()
    
    new_stations = []
    
    for station in stations:
        station_zips = my_station_get_zips(station, distance)
        
        new_pop = station_zips['population'].sum()
        overlap_zips = pd.DataFrame(berkeley_zips.merge(station_zips, right_on='zip', left_on='zip',how='inner'))
        
        overlap_zip_pop = overlap_zips['population_x'].sum()
        
        percent_new = ( new_pop / overlap_zip_pop ) if overlap_zip_pop != 0 else 0
        
        if percent_new < .75:
            new_stations.append(station)
    
    return new_stations

In [63]:
find_net_new_customers(5)

['16th Street Mission',
 '24th Street Mission',
 'Antioch',
 'Balboa Park',
 'Bay Fair',
 'Berryessa',
 'Castro Valley',
 'Civic Center',
 'Colma',
 'Concord',
 'Daly City',
 'Dublin',
 'Embarcadero',
 'Fremont',
 'Glen Park',
 'Hayward',
 'Millbrae',
 'Milpitas',
 'Montgomery Street',
 'North Concord',
 'OAK',
 'Pittsburg',
 'Pittsburg Center',
 'Pleasant Hill',
 'Powell Street',
 'SFO',
 'San Bruno',
 'San Leandro',
 'South Hayward',
 'South San Francisco',
 'Union City',
 'Walnut Creek',
 'Warm Springs',
 'West Dublin']